In [ ]:
# 0) Setup — Facts Creation Gold (GPT-4o-mini)

In [16]:
import ast
import re
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from utils_notebook import SemanticCleaner, extract_answer

load_dotenv()

# Dataset
train_df = pd.read_csv("dataset/arc_challenge_train_processed.csv")

# Modelo único: gpt-4o-mini para tudo
gpt4 = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)

cleaner = SemanticCleaner()

LABELS  = ['A', 'B', 'C', 'D']
number  = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

In [17]:
def format_choices(choices_dict):
    if isinstance(choices_dict, str):
        choices_dict = ast.literal_eval(choices_dict)
    return "\n".join(f"({l}) {t}" for l, t in zip(choices_dict['label'], choices_dict['text']))

def get_choice_text(choices_dict, label):
    if isinstance(choices_dict, str):
        choices_dict = ast.literal_eval(choices_dict)
    idx = number.get(label, 0)
    return choices_dict['text'][idx]

def parse_gold_response(text):
    """Extrai Answer, Reflection e lista de Facts do output do Prompt 1.
    
    Returns: (answer: str, facts: list[str], reflection: str)
    """
    answer_match = re.search(r"Answer\s*:\s*([A-D])", text, re.IGNORECASE)
    refl_match   = re.search(r"Reflection\s*:\s*(.*?)(?=\nFacts?|$)", text, re.IGNORECASE | re.DOTALL)
    facts_match  = re.search(r"Facts?\s*:?\s*\n(.*?)$", text, re.IGNORECASE | re.DOTALL)

    answer     = answer_match.group(1).upper() if answer_match else ""
    reflection = refl_match.group(1).strip()   if refl_match   else ""

    facts = []
    if facts_match:
        lines = facts_match.group(1).split('\n')
        for line in lines:
            clean = line.strip().lstrip('-•*1234567890.)').strip()
            if len(clean.split()) >= 4:
                facts.append(clean)
    facts = facts[:2]  # máximo 2

    return answer, facts, reflection

def parse_score(text):
    """Extrai score (-1, 0 ou 1) do output do Prompt 2."""
    matches = re.findall(r"(?<!\d)-?[01](?!\d)", text)
    if matches:
        sc = int(matches[-1])
        return max(-1, min(1, sc))
    return 0

# Prompts

**Prompt 1** — O modelo responde, explica seu **raciocínio** (Reflection) e extrai os **fatos científicos universais** usados (1 ou 2, preferencialmente 1). Reflexão e fatos são campos separados e independentes.  
**Prompt 2** — Score por fato: curto e direto.

In [23]:
# Prompt 1 — Responde + reflexão do raciocínio + fato(s) científico(s)
answer_fact_template = """Answer the multiple-choice question below.

Question:
{question}

Respond in this exact format:
Answer: [A/B/C/D]
Reflection: [1-2 sentences: the step-by-step reasoning chain that led to the answer]
Facts:
- [one universal scientific principle, max 30 words, no specific objects or names]
- [second principle ONLY if truly independent and essential — otherwise omit this line]"""

answer_fact_prompt = PromptTemplate.from_template(answer_fact_template)
answer_fact_chain  = answer_fact_prompt | gpt4 | StrOutputParser()

In [24]:
# Prompt 2 — Score curto e direto
score_template = """Did this Fact help answer the question correctly?

Q: {question}
Correct: {correct} | Chosen: {chosen} | {outcome}
Fact: {fact}

-1 = fact led to wrong answer | 0 = irrelevant/unused | 1 = essential for correct answer
Score:"""

score_prompt = PromptTemplate.from_template(score_template)
score_chain  = score_prompt | gpt4 | StrOutputParser()

In [25]:
# Testando em uma questão

In [26]:
sample        = train_df.iloc[0]
choices       = ast.literal_eval(sample['choices'])
options_text  = format_choices(choices)
full_question = f"{sample['question']}\n{options_text}"
target        = sample['answerKey']

print(f"Pergunta : {sample['question']}")
print(f"Opções   :\n{options_text}")
print(f"Resposta : {target}")

Pergunta : George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?
Opções   :
(A) dry palms
(B) wet palms
(C) palms covered with oil
(D) palms covered with lotion
Resposta : A


In [27]:
# — Prompt 1: responde + reflexão + fato(s)
raw = answer_fact_chain.invoke({"question": full_question})
print("=== Prompt 1 ===")
print(raw)

pred, facts, reflection = parse_gold_response(raw)
facts      = [cleaner.clean(f) for f in facts if f and len(f.split()) >= 4]
reflection = cleaner.clean(reflection)

# fallback se nenhum fato foi extraído
if not facts:
    facts = [reflection]

is_correct = (pred == target)
status     = "CORRECT" if is_correct else "WRONG"

print(f"\n→ pred={pred} | target={target} | {'✅' if is_correct else '❌'}")
print(f"→ Reflection : {reflection}")
for i, f in enumerate(facts, 1):
    print(f"→ Fact {i}     : {f}")

=== Prompt 1 ===
Answer: A  
Reflection: Rubbing dry palms together generates friction, which produces heat. Wet palms, oil, or lotion can reduce friction and thus decrease heat generation.  
Facts: Friction between surfaces generates heat through kinetic energy conversion.

→ pred=A | target=A | ✅
→ Reflection : Rubbing dry palms together generates friction, which produces heat. Wet palms, oil, or lotion can reduce friction and thus decrease heat generation.
→ Fact 1     : Rubbing dry palms together generates friction, which produces heat. Wet palms, oil, or lotion can reduce friction and thus decrease heat generation.


In [28]:
# — Prompt 2: score por fato
print("=== Prompt 2 ===")
scores = []
for i, fact in enumerate(facts, 1):
    score_raw = score_chain.invoke({
        "question": full_question,
        "correct":  get_choice_text(choices, target),
        "chosen":   get_choice_text(choices, pred),
        "outcome":  status,
        "fact":     fact,
    })
    sc = parse_score(score_raw)
    scores.append(sc)
    print(f"\nFato {i}: {fact}")
    print(f"Resposta modelo: {score_raw.strip()}")
    print(f"→ Score: {sc:+d}")

=== Prompt 2 ===

Fato 1: Rubbing dry palms together generates friction, which produces heat. Wet palms, oil, or lotion can reduce friction and thus decrease heat generation.
Resposta modelo: 1 = essential for correct answer
→ Score: +1


# Criação do Dataset Gold

Processo em **dois prompts por questão**:
1. GPT-4o-mini responde a pergunta, descreve a **reflexão** (raciocínio que levou à resposta) e lista **1 ou 2 fatos científicos universais** (independentes entre si)
2. GPT-4o-mini atribui um **score** (-1 / 0 / 1) para cada fato individualmente

> **Reflexão** = processo de raciocínio do agente (pode mencionar contexto específico da questão)  
> **Fatos** = princípios científicos universais independentes (preferencialmente 1, no máximo 2)

In [29]:
def create_facts_gold_dataset(
    df,
    output_csv="scientific_facts_gold.csv",
    backup_frequency=50,
    start_from=0,
):
    """
    Para cada questão, 2 prompts:
      1. answer_fact_chain  → responde + reflexão de raciocínio + 1 ou 2 fatos científicos
      2. score_chain        → score por fato (-1/0/1)

    Colunas do CSV:
      id, question, choices, correct_answer, model_prediction, is_correct,
      reflection, scientific_fact, quality_score, fact_number, origin
    """
    results = []
    correct_count = 0
    total = len(df)

    loop = tqdm(df.iloc[start_from:].iterrows(),
                total=total - start_from,
                desc="Gold facts")

    for idx, row in loop:
        try:
            q_text        = row['question']
            choices       = ast.literal_eval(row['choices'])
            options_text  = format_choices(choices)
            full_question = f"{q_text}\n{options_text}"
            target        = row['answerKey']

            # — Prompt 1: responde + reflexão + fato(s)
            raw            = answer_fact_chain.invoke({"question": full_question})
            pred, facts, reflection = parse_gold_response(raw)

            # fallback: se o parser não encontrou a letra, tenta extrair_answer
            if pred not in LABELS:
                pred = extract_answer(small_llm_model=gpt4,
                                      model_text_output=raw,
                                      question=full_question)

            reflection = cleaner.clean(reflection)
            facts      = [cleaner.clean(f) for f in facts if f and len(f.split()) >= 4]

            # fallback se nenhum fato foi extraído
            if not facts:
                facts = [reflection]

            is_correct = (pred == target)
            status     = "CORRECT" if is_correct else "WRONG"
            if is_correct:
                correct_count += 1

            # — Prompt 2: score por fato
            for fi, fact in enumerate(facts, 1):
                try:
                    score_raw = score_chain.invoke({
                        "question": full_question,
                        "correct":  get_choice_text(choices, target),
                        "chosen":   get_choice_text(choices, pred),
                        "outcome":  status,
                        "fact":     fact,
                    })
                    score = parse_score(score_raw)
                except Exception:
                    score = 0

                results.append({
                    'id':               row['id'],
                    'question':         q_text,
                    'choices':          str(choices),
                    'correct_answer':   target,
                    'model_prediction': pred,
                    'is_correct':       is_correct,
                    'reflection':       reflection,
                    'scientific_fact':  fact,
                    'quality_score':    score,
                    'fact_number':      fi,
                    'origin':           'train',
                })

            # progresso
            n_done = idx - start_from + 1
            acc    = correct_count / n_done * 100
            loop.set_postfix(acc=f"{acc:.1f}%", facts=len(results), status="✅" if is_correct else "❌")

            # backup periódico
            if n_done % backup_frequency == 0:
                pd.DataFrame(results).to_csv(output_csv, index=False)
                print(f"\n💾 Backup → {output_csv} ({len(results)} registros)")

        except Exception as e:
            print(f"\n❌ Erro Q{idx}: {e}")
            continue

    df_final = pd.DataFrame(results)
    df_final.to_csv(output_csv, index=False)

    n_done = total - start_from
    print(f"\n{'='*60}")
    print(f"✅ CONCLUÍDO")
    print(f"   Questões  : {n_done}")
    print(f"   Acurácia  : {correct_count / n_done * 100:.2f}%")
    print(f"   Registros : {len(results)}")
    print(f"   Arquivo   : {output_csv}")
    print(f"{'='*60}")

    return df_final

In [30]:
df_gold = create_facts_gold_dataset(
    df=train_df,
    output_csv="scientific_facts_gold.csv",
    backup_frequency=25,
    start_from=0,
)

Gold facts:   2%|▏         | 25/1119 [00:57<46:41,  2.56s/it, acc=92.0%, facts=25, status=❌]


💾 Backup → scientific_facts_gold.csv (25 registros)


Gold facts:   4%|▍         | 50/1119 [01:54<41:32,  2.33s/it, acc=94.0%, facts=50, status=✅]


💾 Backup → scientific_facts_gold.csv (50 registros)


Gold facts:   7%|▋         | 75/1119 [02:48<38:25,  2.21s/it, acc=96.0%, facts=75, status=✅]

...


💾 Backup → scientific_facts_gold.csv (1086 registros)


Gold facts:  98%|█████████▊| 1100/1119 [49:13<00:54,  2.88s/it, acc=94.6%, facts=1111, status=✅]


💾 Backup → scientific_facts_gold.csv (1111 registros)


Gold facts: 100%|██████████| 1119/1119 [50:06<00:00,  2.69s/it, acc=94.6%, facts=1130, status=✅]


✅ CONCLUÍDO
   Questões  : 1119
   Acurácia  : 94.64%
   Registros : 1130
   Arquivo   : scientific_facts_gold.csv


In [31]:
df_gold

,id,question,choices,correct_answer,model_prediction,is_correct,reflection,scientific_fact,quality_score,fact_number,origin
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': ['dry palms', 'wet palms', 'palms cov...",A,A,True,"Rubbing dry palms together generates friction,...","Rubbing dry palms together generates friction,...",1,1,train
1,MCAS_2009_5_6516,Which of the following statements best explain...,"{'text': ['The refrigerator door is smooth.', ...",B,B,True,Magnets stick to the refrigerator door because...,Magnets stick to the refrigerator door because...,1,1,train
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': ['cooling of flowing magma.', 'conver...",B,B,True,Folds in sedimentary rock layers are typically...,Folds in sedimentary rock layers are typically...,1,1,train
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': ['worldwide disease', 'global mountai...",D,D,True,The most widely accepted explanation for the m...,Mass extinctions can be caused by sudden envir...,1,1,train
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': ['west', 'east', 'north', 'south'], '...",B,B,True,"The boat is moving northeast, which means the ...","The boat is moving northeast, which means the ...",1,1,train
...,...,...,...,...,...,...,...,...,...,...,...
1125,Mercury_SC_415480,Which change would most likely increase the nu...,"{'text': ['flood', 'drought', 'fire', 'landsli...",A,A,True,Flooding can create new habitats and increase ...,Flooding can create new habitats and increase ...,1,1,train
1126,Mercury_7172795,The skin is the largest organ in the human bod...,"{'text': ['It is made of cells.', 'It acts as ...",C,C,True,Skin is considered an organ because it is made...,Skin is considered an organ because it is made...,1,1,train
1127,NCEOGA_2013_8_59,Which food provides the most energy for the bo...,"{'text': ['potato', 'meat', 'milk', 'fruit'], ...",D,D,True,"Fruits, particularly those high in sugars like...","Fruits, particularly those high in sugars like...",1,1,train
1128,Mercury_7219643,Screech owls have two color variations-red and...,"{'text': ['nesting', 'feeding', 'reproduction'...",D,D,True,The grey screech owl has an advantage in camou...,Camouflage helps animals avoid predation and i...,1,1,train


In [ ]:
print("📋 Primeiros registros:")
print(df_gold[['question', 'model_prediction', 'correct_answer', 'is_correct',
               'scientific_fact', 'reflection', 'quality_score', 'fact_number']].head())

print("\n📊 Distribuição de scores:")
print(df_gold['quality_score'].value_counts().sort_index())
print(f"\nScore médio : {df_gold['quality_score'].mean():.3f}")

print(f"\nAcurácia    : {df_gold['is_correct'].mean()*100:.2f}%")

print(f"\n📊 Fatos por questão:")
print(df_gold.groupby('id')['fact_number'].max().value_counts().sort_index())

print(f"\nTotal registros: {df_gold.shape}")